In [14]:
with open('pg_engine.txt') as f:
    pg_engine = f.readlines()
from sqlalchemy import create_engine
engine = create_engine(pg_engine[0])

In [11]:
import pandas as pd
df = pd.read_sql_query('select * from "rental_listings"',con=engine)

In [12]:
print(df.dtypes)
df.describe()


pid              object
date             object
region           object
neighborhood     object
rent            float64
bedrooms        float64
sqft            float64
rent_sqft       float64
longitude       float64
latitude        float64
county           object
fips_block       object
state            object
dtype: object


,rent,bedrooms,sqft,rent_sqft,longitude,latitude
count,1.499090e+05,149909.000000,149909.000000,149909.000000,149909.000000,149909.000000
mean,3.048307e+03,1.923540,1107.049590,3.868691,-95.139311,36.833912
std,1.248294e+05,0.999565,4535.971404,175.268533,17.255254,5.828810
min,1.000000e+00,0.000000,1.000000,0.000056,-159.530300,2.466618
25%,8.490000e+02,1.000000,750.000000,0.866010,-110.902512,32.822226
50%,1.195000e+03,2.000000,969.000000,1.179412,-92.435923,36.908491
75%,1.700000e+03,2.000000,1210.000000,1.776169,-80.985114,40.768500
max,1.400135e+07,9.000000,999000.000000,30003.500000,0.000000,165.383045


In [ ]:
sfbay = df[df['region']=='sfbay']
sfbay.describe()

In [ ]:
geocoded = pd.read_csv('data/craigslist_data_wblockid.csv', dtype={'GEOID10': object}).rename(columns={'GEOID10':'fips_block'})

In [ ]:
print(geocoded.columns)
geocoded.describe()

In [ ]:
geocoded[:10]

In [ ]:
sfbay_geocoded = geocoded[geocoded['region']=='sfbay']
sfbay_geocoded.describe()